In [2]:
import sqlite3
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import os
import urllib.request
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

cnx = sqlite3.connect('spotify.sqlite')
df = pd.read_sql_query("SELECT id, preview_url FROM tracks", cnx)
df = df[df.preview_url !='']
df = df.tail(-2000000)

                             id  \
2437424  2K5p4xVAUJyHUfvpB8CxSq   
2437425  6B9rbfyhqvgYzM04bl0i55   
2437428  21eh8xwry1icNke3dg1DcY   
2437429  2jjUUKlLlND111RwLTIelB   
2437430  41eYgbaLhjsghK5ATfTkaV   
...                         ...   
8741667  12S1TA13KXl13sCQZgtI5y   
8741668  134MLEMX0AWp5kbMMf90f8   
8741669  13WVJaMn0ztUCU8UFCKQFO   
8741670  11SfAvTgnEEYy4Ef0fyFpE   
8741671  13EW51DKru044OJDJ9Bm8o   

                                               preview_url  
2437424  https://p.scdn.co/mp3-preview/075e5c1aad031f7d...  
2437425  https://p.scdn.co/mp3-preview/7475de012d8f0ef9...  
2437428  https://p.scdn.co/mp3-preview/9e7dc85cbd419639...  
2437429  https://p.scdn.co/mp3-preview/af56e0a6ad7b9e11...  
2437430  https://p.scdn.co/mp3-preview/da76fa90c1c2a852...  
...                                                    ...  
8741667  https://p.scdn.co/mp3-preview/338399db9ca74488...  
8741668  https://p.scdn.co/mp3-preview/3920a3e0e64de14f...  
8741669  https://p.scdn.co/mp3

In [ ]:
print(df)
print(df.count())

In [ ]:


audio_path = '/beegfs/ja62lel/6M/'
processed = 0

#def download():
for index, row in df.iterrows():
    if processed % size == rank:
        uri = row['id']
        url = row['preview_url']
        fn = uri+'.mp3'
        subpath = uri[-2:]
        full_path = os.path.join(audio_path, subpath)
        full_fn = os.path.join(full_path, fn)
        if not os.path.exists(full_path):
            os.makedirs(full_path)
        if not os.path.exists(full_fn):
            for i in range(5):
                try:
                    print("downloading!")
                    print(row['id'] + "|" + row['preview_url'] + "\n")
                    urllib.request.urlretrieve(url, full_fn)
                except:
                    print("Failed to retrieve " + url)
                    continue
                else:
                    break
            else:
                print('url error. continue')
        else:
            print("Exists: skipping")

#download()
print("Done")
